# Ray AIR Lab Solutions

## Basic lab: change XGBoost scaling

In [ ]:
import os
import zipfile
import requests

import ray
from ray import tune
from ray import serve
from ray.air.config import ScalingConfig
from ray.train.xgboost import XGBoostTrainer
from ray.train.xgboost import XGBoostPredictor
from ray.train.batch_predictor import BatchPredictor
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import pandas_read_json
from ray.tune import Tuner, TuneConfig

ray.init()

In [ ]:
trainer = XGBoostTrainer(
    label_column="is_big_tip",
    # num_workers controls parallelism within the training of each model
    scaling_config=ScalingConfig(num_workers=8, use_gpu=False), 
    params={ "objective": "binary:logistic", },
    datasets={"train": train_dataset, "valid": valid_dataset},
)

result = trainer.fit()

In [ ]:
tuner = Tuner(trainer, 
            param_space={'params' : {'max_depth': tune.randint(2, 12)}},
            # num_samples controls how many models are being fit simultaneously across the cluster
            tune_config=TuneConfig(num_samples=8, metric='train-logloss', mode='min'))

checkpoint = tuner.fit().get_best_result().checkpoint

## Intermediate lab: use LightGBM

In [ ]:
from ray.train.lightgbm import LightGBMTrainer

In [ ]:
trainer = LightGBMTrainer(
    label_column="is_big_tip",
    scaling_config=ScalingConfig(num_workers=8, use_gpu=False),
    # the params for LightGBM are similar to those for XGBoost but not identical
    params={ "objective": "binary", },
    datasets={"train": train_dataset, "valid": valid_dataset},
)

result = trainer.fit()

## Advanced lab: use PyTorch

In [ ]:
import torch
import torch.nn as nn
from ray.train.torch import TorchTrainer
from ray.air import session, Checkpoint
from ray import train

In [ ]:
input_size = 6
layer_size = 10
output_size = 1
num_epochs = 1
use_gpu = False

class BasicMLP(nn.Module):
    def __init__(self):
        super(BasicMLP, self).__init__()
        self.layer1 = nn.Linear(input_size, layer_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(layer_size, output_size)

    def forward(self, input):
        return self.layer2(self.relu(self.layer1(input)))

In [ ]:
def train_loop_per_worker():
    dataset_shard = session.get_dataset_shard("train")
    model = BasicMLP()
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

    model = train.torch.prepare_model(model)

    # create list of predictor dimensions
    predictors = [s for s in train_dataset.schema().names if not s.startswith('_')]
    predictors.remove("is_big_tip")

    for epoch in range(num_epochs):
        for batches in dataset_shard.iter_torch_batches(
            batch_size=32, dtypes=torch.float
        ):
            # prepare label matrix
            labels = batches["is_big_tip"].view(-1,1)

            # combine predictor columns into matrix
            inputs = torch.vstack([batches[col] for col in predictors]).t()
            
            output = model(inputs)
            loss = loss_fn(output, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print(f"epoch: {epoch}, loss: {loss.item()}")

        session.report(
            {},
            checkpoint=Checkpoint.from_dict(
                dict(epoch=epoch, model=model.state_dict())
            ),
        )

scaling_config = ScalingConfig(num_workers=3, use_gpu=use_gpu)
trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    scaling_config=scaling_config,
    datasets={"train": train_dataset},
)
result = trainer.fit()